In [18]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import joblib

from data_preprocessing import pipeline
from feature_preprocessing import pipeline2

### 데이터 전처리

In [19]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스2.csv"
# df = pd.read_csv(file_path, encoding ='cp949')
df = pd.read_csv(file_path)

# 데이터 전처리
preprocessing_pipeline = pipeline()
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "250318-03-삭제예정"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_17736\4291326278.py:4: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
c:\Users\user\developments\hk-project\src\data_preprocessing.py:136: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   공고번호               17250 non-null  int64  
 1   주택명                17250 non-null  object 
 2   공급지역코드             17250 non-null  int64  
 3   공급지역명              17250 non-null  object 
 4   공급위치우편번호           17250 non-null  int64  
 5   공급위치               17250 non-null  object 
 6   공급규모               17250 non-null  int64  
 7   모집공고일              17250 non-null  object 
 8   청약접수시작일            17250 non-null  object 
 9   청약접수종료일            17250 non-null  object 
 10  당첨자발표일             17250 non-null  object 
 11  투기과열지구             17250 non-null  object 
 12  조정대상지역             17250 non-null  object 
 13  분양가상한제             17250 non-null  object 
 14  정비사업               17250 non-null  object 
 15  공공주택지구             17250 non-null  object 
 16  대규모택지개발지구          172

### 모델 학습 및 평가

In [21]:
# 전처리된 데이터 가져오기

# file_version = "250317-01"
# file_path = f"./storage/train_data/t``ain-{file_version}.csv"
# df = pd.read_csv(file_path, encoding='cp949')

df = pd.read_csv("./storage/train_data/train-250318-03-삭제예정.csv", encoding = 'cp949')


In [22]:
X = df.drop(columns=["최저당첨가점"])
Y = df["최저당첨가점"]

X_train, X_test, y_train, y_test = train_test_split(
    X,  # 타겟(최저당첨가점) 제외
    Y,  # 타겟
    test_size=0.2,
    random_state=42,
    stratify=df["투기과열지구"]  # 분포 맞추기 위해 미달여부로 stratify
)

# 분양가상한제, 투기과열지구, 조정대상지역 -> 고려해보기염

In [23]:
feature_pipeline = pipeline2()

# 학습 데이터(X_train)에 fit
feature_pipeline.fit(X_train)

# 학습 데이터(X_train)를 변환
X_train_transformed = feature_pipeline.transform(X_train)

# 테스트 데이터(X_test)를 변환
X_test_transformed = feature_pipeline.transform(X_test)

In [24]:
X_train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13800 entries, 16976 to 8817
Data columns (total 61 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   공고번호               13800 non-null  int64  
 1   주택명                13800 non-null  object 
 2   공급지역명              13800 non-null  object 
 3   공급위치우편번호           13800 non-null  int64  
 4   공급위치               13800 non-null  object 
 5   공급규모               13800 non-null  float64
 6   모집공고일              13800 non-null  object 
 7   청약접수시작일            13800 non-null  object 
 8   청약접수종료일            13800 non-null  object 
 9   당첨자발표일             13800 non-null  object 
 10  주택형                13800 non-null  object 
 11  공급세대수              13800 non-null  float64
 12  접수건수               13800 non-null  float64
 13  경쟁률                13800 non-null  float64
 14  최고당첨가점             13765 non-null  object 
 15  평균당첨가점             13765 non-null  object 
 16  기사 번호              13800

In [25]:
# TODO: 여러 모델 학습 및 하이퍼파라미터 최적화

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

# 모델 학습
model.fit(X_train_transformed, y_train)

# 예측
y_pred = model.predict(X_test_transformed)

# 평가 (RMSE와 R^2 계산)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse) # 낮을수록 좋음
print("R^2 Score:", r2) # 1에 가까울수록 좋음

ValueError: could not convert string to float: '호반써밋 목동'

### 모델 저장 

In [7]:
# 모델 저장 및 로드

version = '0.0.1'

# 모델 저장
joblib.dump(model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_0.0.1.pkl']

In [8]:
# 모델 로드
loaded_model = joblib.load("./storage/trained_model/model_0.0.1.pkl")

# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)